In [9]:
import numpy as np
from qutip import *

Question 1

To analyze the security against eavesdropping in the given scenario, let's break down the problem and then proceed with the simulations using QuTiP, a quantum computing simulation library in Python. The key aspect to consider is the quantum bit error rate (QBER) introduced by an eavesdropper (Eve) and how the emission of multiple photons per trial affects the security of the quantum key distribution (QKD) protocol, specifically in this case, a modified version of the B92 protocol.

 Security Analysis:

For a quantum channel with 10% efficiency, the probability of Bob receiving a photon sent by Alice is 0.1. When Alice's source emits two photons (with probability \(p_2\)), there's a risk associated with these multi-photon events: if Eve intercepts the channel, she could measure one photon and forward the other to Bob without being detected under certain strategies. The security of the quantum key distribution is compromised when multi-photon emissions occur because Eve has more information about the key without introducing detectable errors in the polarization states observed by Bob.

The probability \(p_3\) for emitting three or more photons is given as zero, which simplifies our scenario. The security from a spy, in this context, depends on keeping the multi-photon emission rate (i.e., \(p_2\)) at a level where the QBER due to potential eavesdropping activities remains below a certain threshold that allows for secure key distillation after error correction and privacy amplification.

Simulation Setup:

Given \(p_2 = 0.2\), we'll simulate the quantum communication, including the presence and absence of Eve, to observe the effect on the security and performance of the QKD system.

The simulation involves creating a series of functions to mimic the actions of Alice, Bob, and Eve under the conditions specified. Since QuTiP isn't accessible here, I'll outline the steps you would typically follow in a Python environment where QuTiP is installed:

1. Alice's Function:
   - Generates a random bit (0 or 1).
   - Depending on the bit value, chooses a corresponding polarization state for the photon(s) she sends.
   - Emits 1 photon with probability \(1 - p_2\) and 2 photons with probability \(p_2\).

2. Eve's Function:
   - If Eve is present, she intercepts the photon(s). Given that Eve knows the number of photons but does not collapse the polarization state, she could, for example, measure one photon and forward the other without any interaction (in the case of 2 photons being sent).
   - Develop a strategy for Eve based on the number of photons she detects.

3. Bob's Function:
   - Attempts to receive and measure the photon(s) sent by Alice with a 10% chance of success per photon, simulating the channel efficiency.
   - Since Bob does not have a number-resolving photon detector, he does not know the number of photons and measures the polarization state to infer the bit.

Conducting the Simulation:

For each trial:
- Generate Alice's bit and the number of photons.
- Pass the photons through Eve and simulate any potential eavesdropping activities.
- Determine if Bob successfully receives and correctly interprets the bit.

To analyze security, compare the success rates and error rates of Bob's bit reception with and without Eve's interference. Specifically, you would look at how the presence of multi-photon pulses affects the security of the communication. A higher success rate without Eve compared to with Eve indicates that the protocol has some resilience to eavesdropping, but any significant discrepancies could highlight security vulnerabilities, particularly related to multi-photon emissions.


a) Without using numpy and random number generator

In [10]:


def alice(p2, num_bits):
    photon_numbers = np.random.choice([1, 2], size=num_bits, p=[1 - p2, p2])
    polarizations = np.random.choice(['H', 'V'], size=num_bits)
    return list(zip(photon_numbers, polarizations))

def eve(photons, eavesdropping_probability):
    # Eve can choose to intercept or not based on the eavesdropping probability
    intercepted_photons = []
    for photon in photons:
        if np.random.rand() < eavesdropping_probability:
            # Intercepted photon
            intercepted_photons.append(photon)
        else:
            # Pass the photon without interception
            intercepted_photons.append(photon)
    return intercepted_photons

def bob(photons):
    # Bob only receives 10% of the events from Alice due to channel inefficiency
    received_photons = []
    for photon in photons:
        if np.random.rand() < 0.1:  # 10% efficiency
            received_photons.append(photon)
    return received_photons

def simulate(p2, num_bits, eavesdropping_probability=0):
    # Alice sends photons
    alice_photons = alice(p2, num_bits)
    
    # Eve intercepts photons if present
    if eavesdropping_probability > 0:
        intercepted_photons = eve(alice_photons, eavesdropping_probability)
    else:
        intercepted_photons = alice_photons
    
    # Bob receives photons
    bob_photons = bob(intercepted_photons)
    
    return alice_photons, intercepted_photons, bob_photons

# Testing the simulation
p2 = 0.2
num_bits = 1000

# Without Eve
alice_photons, _, bob_photons_without_eve = simulate(p2, num_bits)

# With Eve
eavesdropping_probability = 1  # Eve receives and transmits with unit efficiency
_, intercepted_photons, bob_photons_with_eve = simulate(p2, num_bits, eavesdropping_probability)

# Count how many bits Bob received correctly
correct_bits_without_eve = sum([1 for alice_photon, bob_photon in zip(alice_photons, bob_photons_without_eve) if alice_photon[1] == bob_photon[1]])
correct_bits_with_eve = sum([1 for alice_photon, bob_photon in zip(alice_photons, bob_photons_with_eve) if alice_photon[1] == bob_photon[1]])

print("Bits received by Bob without Eve:", correct_bits_without_eve)
print("Bits received by Bob with Eve:", correct_bits_with_eve)


Bits received by Bob without Eve: 47
Bits received by Bob with Eve: 52


Using polarization and qutip

In [46]:

# Parameters
p2 = 0.2  # Probability of emitting 2 photons
channel_efficiency = 0.1  # Channel efficiency (probability of photon reaching Bob)

# Function to simulate Alice's behavior
def alice_emits(p2):
    bit = np.random.choice([0, 1])  # Alice's bit choice
    photons = np.random.choice([1, 2], p=[1 - p2, p2])  # Number of photons emitted
    
    # Polarization states for B92: |0> for bit 0 and |+> for bit 1
    if bit == 0:
        state = basis(2, 0)  # |0>
    else:
        state = (basis(2, 0) + basis(2, 1)).unit()  # |+>
    
    return bit, photons, state

# Function to simulate Eve's behavior
def eve_intercepts(photons, state):
    # Assuming Eve knows the number of photons but does not alter the state
    # This is a placeholder for any strategy Eve might employ
    return photons, state  # ﬁ for this example

# Function to simulate Bob's measurement
def bob_receives(photons, state):
    received = np.random.binomial(1, channel_efficiency * photons)  # Simulate channel efficiency
    if received:
        # Bob's measurement basis choice: |0> and |+>
        measurement_basis = np.random.choice(["0", "+"])
        if measurement_basis == "0":
            measure_op = basis(2, 0) * basis(2, 0).dag()  # Projector for |0>
        else:
            measure_op = (basis(2, 0) + basis(2, 1)).unit() * (basis(2, 0) + basis(2, 1)).unit().dag()  # Projector for |+>
        
        # Measurement result
        result = (state.dag() * measure_op * state).norm()
        return 1 if result > 0.5 else 0  # Detected the bit
    return None  # No photon received

# Simulation parameters
trials = 10000
success_without_eve = 0
success_with_eve = 0

for _ in range(trials):
    bit, photons, state = alice_emits(p2)
    # Without Eve
    bob_result = bob_receives(photons, state)
    if bob_result is not None and bob_result == bit:
        success_without_eve += 1
    
    # With Eve
    photons_eve, state_eve = eve_intercepts(photons, state)
    bob_result_eve = bob_receives(photons_eve, state_eve)
    if bob_result_eve is not None and bob_result_eve == bit:
        success_with_eve += 1

success_rate_without_eve = success_without_eve / trials
success_rate_with_eve = success_with_eve / trials

#success_rate_without_eve, success_rate_with_eve
print("\n Success rate without Eve",success_rate_without_eve)
print("\n Success rate with Eve", success_rate_with_eve)



 Success rate without Eve 0.0602

 Success rate with Eve 0.0587


In [45]:

def alice_emit_photons(p2, num_trials):
    # Alice emits photons according to probabilities p1 and p2
    photon_numbers = np.zeros(num_trials, dtype=int)
    for i in range(num_trials):
        # Randomly select the number of photons emitted
        if random() < p2:
            photon_numbers[i] = 2
        else:
            photon_numbers[i] = 1
    return photon_numbers

def eve_intercept(photon_numbers):
    # Eve intercepts photons
    intercepted_photons = []
    for num in photon_numbers:
        # Eve's strategy can be implemented here, for simplicity let's assume she just copies the state
        intercepted_photons.append(num)
    return intercepted_photons

def bob_measure(photon_numbers, efficiency):
    # Bob receives photons considering channel efficiency
    received_photons = []
    for num in photon_numbers:
        if random() < efficiency:
            received_photons.append(num)
    return received_photons

# Parameters
p2 = 0.2  # Probability of emitting 2 photons
efficiency = 0.1  # Channel efficiency
num_trials = 1000

# Alice emits photons
photon_numbers = alice_emit_photons(p2, num_trials)

# Eve intercepts photons
intercepted_photons = eve_intercept(photon_numbers)

# Bob receives photons
received_photons = bob_measure(intercepted_photons, efficiency)

# Compare the output of Bob with and without Eve present
print("Without Eve:")
print("Number of photons received by Bob:", len(received_photons))
print("With Eve:")
print("Number of photons intercepted by Eve:", len(intercepted_photons))


Without Eve:
Number of photons received by Bob: 90
With Eve:
Number of photons intercepted by Eve: 1000


To analyze security, compare the success rates and error rates (if applicable) of Bob's bit reception with and without Eve's interference. Specifically, you would look at how the presence of multi-photon pulses affects the security of the communication. A higher success rate without Eve compared to with Eve indicates that the protocol has some resilience to eavesdropping, but any significant discrepancies could highlight security vulnerabilities, particularly related to multi-photon emissions.

b) Using classical random genrator

In [12]:
# Testing the simulation
p2 = 0.05

# Without Eve
alice_photons, _, bob_photons_without_eve = simulate(p2, num_bits)

# With Eve
_, intercepted_photons, bob_photons_with_eve = simulate(p2, num_bits, eavesdropping_probability)

# Count how many bits Bob received correctly
correct_bits_without_eve = sum([1 for alice_photon, bob_photon in zip(alice_photons, bob_photons_without_eve) if alice_photon[1] == bob_photon[1]])
correct_bits_with_eve = sum([1 for alice_photon, bob_photon in zip(alice_photons, bob_photons_with_eve) if alice_photon[1] == bob_photon[1]])

print("Bits received by Bob without Eve:", correct_bits_without_eve)
print("Bits received by Bob with Eve:", correct_bits_with_eve)


Bits received by Bob without Eve: 52
Bits received by Bob with Eve: 57


Using polarization and qutip

In [47]:

# Parameters
p2 = 0.05  # Probability of emitting 2 photons
channel_efficiency = 0.1  # Channel efficiency (probability of photon reaching Bob)

# Function to simulate Alice's behavior
def alice_emits(p2):
    bit = np.random.choice([0, 1])  # Alice's bit choice
    photons = np.random.choice([1, 2], p=[1 - p2, p2])  # Number of photons emitted
    
    # Polarization states for B92: |0> for bit 0 and |+> for bit 1
    if bit == 0:
        state = basis(2, 0)  # |0>
    else:
        state = (basis(2, 0) + basis(2, 1)).unit()  # |+>
    
    return bit, photons, state

# Function to simulate Eve's behavior
def eve_intercepts(photons, state):
    # Assuming Eve knows the number of photons but does not alter the state
    # This is a placeholder for any strategy Eve might employ
    return photons, state  # Simply forwards the state without alteration for this example

# Function to simulate Bob's measurement
def bob_receives(photons, state):
    received = np.random.binomial(1, channel_efficiency * photons)  # Simulate channel efficiency
    if received:
        # Bob's measurement basis choice: |0> and |+>
        measurement_basis = np.random.choice(["0", "+"])
        if measurement_basis == "0":
            measure_op = basis(2, 0) * basis(2, 0).dag()  # Projector for |0>
        else:
            measure_op = (basis(2, 0) + basis(2, 1)).unit() * (basis(2, 0) + basis(2, 1)).unit().dag()  # Projector for |+>
        
        # Measurement result
        result = (state.dag() * measure_op * state).norm()
        return 1 if result > 0.5 else 0  # Detected the bit
    return None  # No photon received

# Simulation parameters
trials = 10000
success_without_eve = 0
success_with_eve = 0

for _ in range(trials):
    bit, photons, state = alice_emits(p2)
    # Without Eve
    bob_result = bob_receives(photons, state)
    if bob_result is not None and bob_result == bit:
        success_without_eve += 1
    
    # With Eve
    photons_eve, state_eve = eve_intercepts(photons, state)
    bob_result_eve = bob_receives(photons_eve, state_eve)
    if bob_result_eve is not None and bob_result_eve == bit:
        success_with_eve += 1

success_rate_without_eve = success_without_eve / trials
success_rate_with_eve = success_with_eve / trials

print("\n Success rate without Eve",success_rate_without_eve)
print("\n Success rate with Eve", success_rate_with_eve)



 Success rate without Eve 0.0498

 Success rate with Eve 0.055


In [65]:

def alice_emit_photons(p2, num_trials):
    # Alice emits photons according to probabilities p1 and p2
    photon_numbers = np.zeros(num_trials, dtype=int)
    for i in range(num_trials):
        # Randomly select the number of photons emitted
        if random() < p2:
            photon_numbers[i] = 2
        else:
            photon_numbers[i] = 1
    return photon_numbers

def eve_intercept(photon_numbers):
    # Eve intercepts photons
    intercepted_photons = []
    for num in photon_numbers:
        # Eve's strategy can be implemented here, for simplicity let's assume she just copies the state
        intercepted_photons.append(num)
    return intercepted_photons

def bob_measure(photon_numbers, efficiency):
    # Bob receives photons considering channel efficiency
    received_photons = []
    for num in photon_numbers:
        if random() < efficiency:
            received_photons.append(num)
    return received_photons

# Parameters
p2 = 0.05  # Probability of emitting 2 photons
efficiency = 0.1  # Channel efficiency
num_trials = 1000

# Alice emits photons
photon_numbers = alice_emit_photons(p2, num_trials)

# Eve intercepts photons
intercepted_photons = eve_intercept(photon_numbers)

# Bob receives photons
received_photons = bob_measure(intercepted_photons, efficiency)

# Compare the output of Bob with and without Eve present
print("Without Eve:")
print("Number of photons received by Bob:", len(received_photons))
print("With Eve:")
print("Number of photons intercepted by Eve:", len(intercepted_photons))


Without Eve:
Number of photons received by Bob: 98
With Eve:
Number of photons intercepted by Eve: 1000


Question 2)

In [66]:


# Parameters
p2 = 0.2  # Probability of emitting 2 photons for signal states
p_decoy = 0.1  # Probability that a given pulse is a decoy
channel_efficiency = 0.1  # Channel efficiency

# Function to simulate Alice's behavior with decoy states
def alice_emits(p2, p_decoy):
    bit = np.random.choice([0, 1])  # Alice's bit choice for signal states
    is_decoy = np.random.choice([True, False], p=[p_decoy, 1 - p_decoy])  # Decoy or signal
    if is_decoy:
        # Decoy state: could be empty or weak (0 or 1 photon)
        photons = np.random.choice([0, 1])
        state = None if photons == 0 else (basis(2, 0) + basis(2, 1)).unit()  # Weak decoy state
    else:
        # Signal state
        photons = np.random.choice([1, 2], p=[1 - p2, p2])
        state = basis(2, 0) if bit == 0 else (basis(2, 0) + basis(2, 1)).unit()
    
    return bit, photons, state, is_decoy
print(bit, photons, state, is_decoy)

# Modifications to Eve and Bob's functions would be minimal; mainly, Bob needs to handle the possibility of receiving no photon

# After communication, Alice reveals which pulses were decoys
# Alice and Bob analyze the decoy statistics to check for eavesdropping

# Simulation parameters and loop similar to the original, with added logic for handling decoy states


0 1 Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[1.]
 [0.]] False


In [21]:

p2 = 0.2  # Probability of emitting 2 photons
channel_efficiency = 0.1  # Channel efficiency
decoy_probability = 0.3  # Probability of emitting a decoy pulse

# Function to simulate Alice's behavior including decoy states
def alice_emits(p2, decoy_probability):
    bit = np.random.choice([0, 1])  # Alice's bit choice
    photons = np.random.choice([1, 2], p=[1 - p2, p2])  # Number of photons emitted
    decoy = np.random.choice([True, False], p=[decoy_probability, 1 - decoy_probability])  # Decoy or signal pulse
    
    # Polarization states for B92: |0> for bit 0 and |+> for bit 1
    if bit == 0:
        state = basis(2, 0)  # |0>
    else:
        state = (basis(2, 0) + basis(2, 1)).unit()  # |+>
    
    return bit, photons, state, decoy

# Updated function to simulate Eve's behavior considering decoy states
def eve_intercepts(photons, state, decoy):
    
    return photons, state, decoy

# Function to simulate Bob's measurement including decoy state processing
def bob_receives(photons, state):
    received = np.random.binomial(1, channel_efficiency * photons)  # Simulate channel efficiency
    if received:
        measurement_basis = np.random.choice(["0", "+"])
        if measurement_basis == "0":
            measure_op = basis(2, 0) * basis(2, 0).dag()  # Projector for |0>
        else:
            measure_op = (basis(2, 0) + basis(2, 1)).unit() * (basis(2, 0) + basis(2, 1)).unit().dag()  # Projector for |+>
        
        result = (state.dag() * measure_op * state).norm()
        return 1 if result > 0.5 else 0  # Detected the bit
    return None  # No photon received

# Simulation parameters
trials = 10000
success_without_eve = 0
success_with_eve = 0
decoy_trials = 100
decoy_success = 0

for _ in range(trials):
    bit, photons, state, decoy = alice_emits(p2, decoy_probability)
    # Without Eve
    bob_result = bob_receives(photons, state)
    if bob_result is not None and bob_result == bit and not decoy:
        success_without_eve += 1
    elif bob_result is not None and decoy:
        # Count decoy trials separately
        decoy_trials += 1
        if bob_result == bit:
            decoy_success += 1
    
    # With Eve
    photons_eve, state_eve, decoy_eve = eve_intercepts(photons, state, decoy)
    bob_result_eve = bob_receives(photons_eve, state_eve)
    if bob_result_eve is not None and bob_result_eve == bit and not decoy_eve:
        success_with_eve += 1

success_rate_without_eve = success_without_eve / (trials - decoy_trials)
success_rate_with_eve = success_with_eve / (trials - decoy_trials)
decoy_detection_rate = decoy_success / decoy_trials

print("\n Success rate without Eve",success_rate_without_eve)
print("\n Success rate with Eve", success_rate_with_eve)
print("\n Decoy detection rate", decoy_detection_rate)



 Success rate without Eve 0.04319563849863703

 Success rate with Eve 0.046550639547074855

 Decoy detection rate 0.38961038961038963


In [44]:

# Decoy state parameters for the second part of the simulation
p_decoy = 0.1  # Probability of Alice choosing to send a decoy state
decoy_photons_distribution = [1]  # Decoys are weak pulses, here we choose only 1 photon for simplicity

# Functions
def alice_emits(p2, is_decoy=False):
    bit = np.random.choice([0, 1])  # Alice's bit choice
    if is_decoy:
        photons = np.random.choice(decoy_photons_distribution)
        state = None  # Decoys do not encode a bit
    else:
        photons = np.random.choice([1, 2], p=[1 - p2, p2])  # Number of photons emitted
        if bit == 0:
            state = basis(2, 0)  # |0>
        else:
            state = (basis(2, 0) + basis(2, 1)).unit()  # |+>
    return bit, photons, state, is_decoy

def bob_receives(photons, state, channel_efficiency):
    received = np.random.binomial(1, channel_efficiency * photons)  # Simulate channel efficiency
    if received and state is not None:
        measurement_basis = np.random.choice(["0", "+"])
        if measurement_basis == "0":
            measure_op = basis(2, 0) * basis(2, 0).dag()
        else:
            measure_op = (basis(2, 0) + basis(2, 1)).unit() * (basis(2, 0) + basis(2, 1)).unit().dag()
        result = (state.dag() * measure_op * state).norm()
        return 1 if result > 0.5 else 0  # Detected the bit
    return None  # No photon received or decoy state

# Simulation without decoy states
np.random.seed(42)  # For reproducibility
trials = 10000
success_without_eve_no_decoy = 0

for _ in range(trials):
    bit, photons, state, _ = alice_emits(p2)
    bob_result = bob_receives(photons, state, channel_efficiency)
    if bob_result is not None and bob_result == bit:
        success_without_eve_no_decoy += 1

efficiency_no_decoy = success_without_eve_no_decoy / trials

# Simulation with decoy states
success_without_eve_with_decoy = 0
non_decoy_trials = 0

for _ in range(trials):
    is_decoy = np.random.choice([True, False], p=[p_decoy, 1 - p_decoy])
    bit, photons, state, _ = alice_emits(p2, is_decoy)
    if not is_decoy:
        non_decoy_trials += 1
        bob_result = bob_receives(photons, state, channel_efficiency)
        if bob_result is not None and bob_result == bit:
            success_without_eve_with_decoy += 1

efficiency_with_decoy = success_without_eve_with_decoy / non_decoy_trials


efficiency_no_decoy, efficiency_with_decoy

print("Efficiency without decoy states:", efficiency_no_decoy)
print("Efficiency with decoy states:", efficiency_with_decoy)


Efficiency without decoy states: 0.0622
Efficiency with decoy states: 0.05822756624359831


Extra tries to solve the program, just in case the question is expecting more...

In [56]:


# Parameters
alpha_data = 0.5  # Amplitude for data pulses
alpha_decoy = 0.1  # Amplitude for decoy pulses
num_bits = 100  # Number of bits in the simulation

# Quantum states for H and V polarizations
ket_H = basis(2, 0)
ket_V = basis(2, 1)

# Function to prepare quantum states
def prepare_states(num_bits, alpha_data, alpha_decoy):
    states = []
    for _ in range(num_bits):
        if np.random.rand() < 0.5:
            # Prepare data pulse
            alpha = alpha_data
            polarization = ket_H if np.random.rand() < 0.5 else ket_V
        else:
            # Prepare decoy pulse
            alpha = alpha_decoy
            polarization = ket_H if np.random.rand() < 0.5 else ket_V
        # Using coherent states as an example; in practice, could be single-photon states
        state = tensor(coherent(2, alpha), polarization)
        states.append(state)
    return states


states = prepare_states(num_bits, alpha_data, alpha_decoy)
print(states[0])
print(states[1])


Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.87758256]
 [0.        ]
 [0.47942554]
 [0.        ]]
Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.        ]
 [0.87758256]
 [0.        ]
 [0.47942554]]


In [64]:

def alice_emit_photons(p2, p_decoy, num_trials):
    photon_numbers = np.zeros(num_trials, dtype=int)
    pulse_types = np.zeros(num_trials, dtype=int)  # 0 for data, 1 for decoy
    for i in range(num_trials):
        if random() < p_decoy:
            pulse_types[i] = 1  # Decoy pulse
            photon_numbers[i] = 1  # Assume decoy pulses are single-photon for simplicity
        else:
            pulse_types[i] = 0  # Data pulse
            photon_numbers[i] = 2 if random() < p2 else 1
    return photon_numbers, pulse_types

def bob_measure(photon_numbers, pulse_types, efficiency):
    received_photons = []
    for i in range(len(photon_numbers)):
        if random() < efficiency:
            received_photons.append((photon_numbers[i], pulse_types[i]))
    return received_photons

def simulation(num_trials, p2, p_decoy, efficiencies):
    results = []
    for efficiency in efficiencies:
        photon_numbers, pulse_types = alice_emit_photons(p2, p_decoy, num_trials)
        received_photons = bob_measure(photon_numbers, pulse_types, efficiency)
        decoy_detected = sum(1 for _, pulse_type in received_photons if pulse_type == 1)
        data_detected = sum(1 for _, pulse_type in received_photons if pulse_type == 0)
        results.append((efficiency, decoy_detected, data_detected))
    return results

# Parameters
p2 = 0.2  # Probability of emitting 2 photons
p_decoy = 0.1  # Probability of emitting a decoy pulse
num_trials = 10000
efficiencies = np.linspace(0.05, 1.0, 20)  # Range of channel efficiencies to simulate

# Run simulation
results = simulation(num_trials, p2, p_decoy, efficiencies)

# Display results
for efficiency, decoy_detected, data_detected in results:
    print(f"Efficiency: {efficiency:.2f}, Decoy Detected: {decoy_detected}, Data Detected: {data_detected}")



Efficiency: 0.05, Decoy Detected: 42, Data Detected: 438
Efficiency: 0.10, Decoy Detected: 103, Data Detected: 938
Efficiency: 0.15, Decoy Detected: 151, Data Detected: 1313
Efficiency: 0.20, Decoy Detected: 211, Data Detected: 1783
Efficiency: 0.25, Decoy Detected: 265, Data Detected: 2284
Efficiency: 0.30, Decoy Detected: 328, Data Detected: 2696
Efficiency: 0.35, Decoy Detected: 354, Data Detected: 3166
Efficiency: 0.40, Decoy Detected: 408, Data Detected: 3649
Efficiency: 0.45, Decoy Detected: 430, Data Detected: 4072
Efficiency: 0.50, Decoy Detected: 513, Data Detected: 4475
Efficiency: 0.55, Decoy Detected: 528, Data Detected: 5041
Efficiency: 0.60, Decoy Detected: 587, Data Detected: 5430
Efficiency: 0.65, Decoy Detected: 670, Data Detected: 5882
Efficiency: 0.70, Decoy Detected: 675, Data Detected: 6279
Efficiency: 0.75, Decoy Detected: 791, Data Detected: 6691
Efficiency: 0.80, Decoy Detected: 779, Data Detected: 7167
Efficiency: 0.85, Decoy Detected: 860, Data Detected: 7631
